In [2]:
%cd /Users/johannesbreit/Library/CloudStorage/OneDrive-UniversityCollegeLondon/COMP0087/project/nlpproject

/Users/johannesbreit/Library/CloudStorage/OneDrive-UniversityCollegeLondon/COMP0087/project/nlpproject


In [3]:
import pandas as pd
import numpy as np
import re
import random
import os
import gzip
from collections import Counter, defaultdict, namedtuple

In [4]:
pwd

'/Users/johannesbreit/Library/CloudStorage/OneDrive-UniversityCollegeLondon/COMP0087/project/nlpproject'

In [5]:
ls

README.md         nlp/              requirements.txt


In [6]:
corpus_in = pd.read_csv('../nlpproject/nlp/data/example_inputs_pos_and_neg.tsv.gz', compression='gzip', header=0, sep='\t')
corpus_in.head()

,entity1,entity2,left,middle,right,article,label
0,Apple Inc,Samsung Electronics Co Ltd,\nThese reviews have been especially popular w...,(AAPL.O) and,(005930.KS) frequently hit with patent infrin...,50b8f1a67bb7c2cab2fe56b55a24f59ad1703c07b7053e...,0
1,Bharti Airtel Ltd,Reliance Industries Ltd,* Q4 profit 829 mln rupees vs. 3.73 bln rupees...,posted its lowest quarterly profit in nearly ...,"Jio, the telecoms arm of Reliance, into India...",603eeb451da67a70850a823e4eb4e1221a5358040bde3b...,0
2,Reliance Industries Ltd,Bharti Airtel Ltd,"\nThe entry of Reliance Jio, the telecoms arm of",", into India's telecoms sector has set off a b...",'s managing director and chief executive for I...,603eeb451da67a70850a823e4eb4e1221a5358040bde3b...,0
3,Bharti Airtel Ltd,Reliance Industries Ltd,The U.S. Supreme Court on Tuesday ruled that f...,dan-based Arab Bank Plc helped finance militan...,"24, 2016.",26cb6ec3607fdb028b712d260131058e9b01b9aadc26cf...,1
4,Reliance Industries Ltd,Bharti Airtel Ltd,REUTERS/Aziz Ta,Photo\nThe 5-4 decision brought to an end a l...,minor connection between the terrorist attacks...,26cb6ec3607fdb028b712d260131058e9b01b9aadc26cf...,1


In [7]:
#Remove any \n from the rows 
corpus_in.replace("\n","",inplace=True)
corpus_in.replace('(\n)','',regex=True,inplace=True)
corpus_in.head()

,entity1,entity2,left,middle,right,article,label
0,Apple Inc,Samsung Electronics Co Ltd,These reviews have been especially popular wit...,(AAPL.O) and,(005930.KS) frequently hit with patent infrin...,50b8f1a67bb7c2cab2fe56b55a24f59ad1703c07b7053e...,0
1,Bharti Airtel Ltd,Reliance Industries Ltd,* Q4 profit 829 mln rupees vs. 3.73 bln rupees...,posted its lowest quarterly profit in nearly ...,"Jio, the telecoms arm of Reliance, into India...",603eeb451da67a70850a823e4eb4e1221a5358040bde3b...,0
2,Reliance Industries Ltd,Bharti Airtel Ltd,"The entry of Reliance Jio, the telecoms arm of",", into India's telecoms sector has set off a b...",'s managing director and chief executive for I...,603eeb451da67a70850a823e4eb4e1221a5358040bde3b...,0
3,Bharti Airtel Ltd,Reliance Industries Ltd,The U.S. Supreme Court on Tuesday ruled that f...,dan-based Arab Bank Plc helped finance militan...,"24, 2016.",26cb6ec3607fdb028b712d260131058e9b01b9aadc26cf...,1
4,Reliance Industries Ltd,Bharti Airtel Ltd,REUTERS/Aziz Ta,PhotoThe 5-4 decision brought to an end a law...,minor connection between the terrorist attacks...,26cb6ec3607fdb028b712d260131058e9b01b9aadc26cf...,1


In [8]:
#Create sentences
df_data_sentences = corpus_in[['left', 'entity1', "middle",'entity2','right']].astype(str).agg(' '.join, axis=1)
corpus_in['sentences'] = df_data_sentences.values
corpus_in.head()

,entity1,entity2,left,middle,right,article,label,sentences
0,Apple Inc,Samsung Electronics Co Ltd,These reviews have been especially popular wit...,(AAPL.O) and,(005930.KS) frequently hit with patent infrin...,50b8f1a67bb7c2cab2fe56b55a24f59ad1703c07b7053e...,0,These reviews have been especially popular wit...
1,Bharti Airtel Ltd,Reliance Industries Ltd,* Q4 profit 829 mln rupees vs. 3.73 bln rupees...,posted its lowest quarterly profit in nearly ...,"Jio, the telecoms arm of Reliance, into India...",603eeb451da67a70850a823e4eb4e1221a5358040bde3b...,0,* Q4 profit 829 mln rupees vs. 3.73 bln rupees...
2,Reliance Industries Ltd,Bharti Airtel Ltd,"The entry of Reliance Jio, the telecoms arm of",", into India's telecoms sector has set off a b...",'s managing director and chief executive for I...,603eeb451da67a70850a823e4eb4e1221a5358040bde3b...,0,"The entry of Reliance Jio, the telecoms arm of..."
3,Bharti Airtel Ltd,Reliance Industries Ltd,The U.S. Supreme Court on Tuesday ruled that f...,dan-based Arab Bank Plc helped finance militan...,"24, 2016.",26cb6ec3607fdb028b712d260131058e9b01b9aadc26cf...,1,The U.S. Supreme Court on Tuesday ruled that f...
4,Reliance Industries Ltd,Bharti Airtel Ltd,REUTERS/Aziz Ta,PhotoThe 5-4 decision brought to an end a law...,minor connection between the terrorist attacks...,26cb6ec3607fdb028b712d260131058e9b01b9aadc26cf...,1,REUTERS/Aziz Ta Reliance Industries Ltd Photo...


In [9]:
kb_df = pd.read_csv('../nlpproject/nlp/data/example_kb_all_supplier.tsv.gz', compression='gzip', 
                                 names= ['relation', 'subject', 'object'], sep='\t', quotechar='"')

In [10]:
kb_df.head()

,relation,subject,object
0,supplier of,Apple Inc,Seoul Semiconductor Co Ltd
1,supplier of,Koizumi furnitech Thailand Co Ltd,Seoul Semiconductor Co Ltd
2,supplier of,Amazon.com Inc,DS Smith PLC
3,supplier of,Procter & Gamble Co,DS Smith PLC
4,supplier of,Zymeworks Inc,DS Smith PLC


In [13]:
def stanford_split(kb, corpus, split_ratios = [0.6, 0.2, 0.2], verbose = False):
    
    assert sum(split_ratios) == 1, 'Split Ratios do not add up to 1'

    # Randomly split the subjects in the KB between train/dev/test splits 
    # TODO: Random number generation split does not create exact splits - make this exact with length split instead (but works for now)
    def subject_split(kb, split_ratios):
        subjects = pd.DataFrame(kb.subject.unique()).rename(columns = {0: 'subject'})
        subjects['split'] = [np.random.choice(3, p =  split_ratios) for _ in range(len(subjects))] 
        return subjects

    if verbose: pass

    # Merge split numbers for subjects (E1) back onto kb df
    kb_with_splits = kb_df.merge(subject_split(kb, split_ratios), how = 'left', on = 'subject')

    split_dict = {sbj: split for sbj, split in zip(kb_with_splits.subject, kb_with_splits.split)}

    corpus_with_splits = corpus.copy()

    # Use stanford Rule:
    # If E1 in corpus in a KB split -> place sentence to that split.
    # Else if E2 in corpus in a KB split -> place sentance to that split.
    # If neither of above, assign sentence to random split.
    splits = []
    count_not_matched = 0
    for E1, E2 in zip(corpus_with_splits.entity1, corpus_with_splits.entity2): 
        if E1 in split_dict.keys():
            splits.append(split_dict[E1])
        elif E2 in split_dict.keys():
            splits.append(split_dict[E2])
        else:
            splits.append(np.random.choice(3, p = split_ratios))
            if verbose: print('{}, {} not in KB split' .format(E1, E2))
            count_not_matched += 1
    print('Total of {} entries in corpus not matched to KB. \n' .format(count_not_matched))
    corpus_with_splits['split'] = splits

    # if verbose:
    print('KB split ratios: \n {} \n' .format(kb_with_splits.split.value_counts(normalize = True)))
    print('Corpus split ratios: \n {} \n' .format(corpus_with_splits.split.value_counts(normalize = True)))

    return kb_with_splits, corpus_with_splits


In [14]:
kb_split, corpus_split = stanford_split(kb_df, corpus_in)

Total of 38 entries in corpus not matched to KB. 

KB split ratios: 
 0    0.575583
1    0.213867
2    0.210550
Name: split, dtype: float64 

Corpus split ratios: 
 0    0.546038
1    0.262683
2    0.191279
Name: split, dtype: float64 



In [15]:
kb_split

,relation,subject,object,split
0,supplier of,Apple Inc,Seoul Semiconductor Co Ltd,0
1,supplier of,Koizumi furnitech Thailand Co Ltd,Seoul Semiconductor Co Ltd,1
2,supplier of,Amazon.com Inc,DS Smith PLC,1
3,supplier of,Procter & Gamble Co,DS Smith PLC,0
4,supplier of,Zymeworks Inc,DS Smith PLC,0
...,...,...,...,...
18081,supplier of,Cisco Systems Inc (Pre-Merger),Synetrix (Holdings) Ltd,0
18082,supplier of,Cisco Systems Inc (Pre-Merger),Servcorp Ltd,0
18083,supplier of,Cisco Systems Inc (Pre-Merger),SolarWinds Corp,0
18084,supplier of,Cisco Systems Inc (Pre-Merger),Unified Communications Ltd,0


In [16]:
kb_split.split.value_counts(normalize = False), kb_split.split.value_counts(normalize = True)

(0    10410
 1     3868
 2     3808
 Name: split, dtype: int64,
 0    0.575583
 1    0.213867
 2    0.210550
 Name: split, dtype: float64)

In [17]:
corpus_split.split.value_counts(), corpus_split.split.value_counts(normalize=True)

(0    47122
 1    22669
 2    16507
 Name: split, dtype: int64,
 0    0.546038
 1    0.262683
 2    0.191279
 Name: split, dtype: float64)

In [18]:
kb_split[kb_split.subject.astype(str).str.contains('Apple')]

,relation,subject,object,split
0,supplier of,Apple Inc,Seoul Semiconductor Co Ltd,0
44,supplier of,Apple Inc,TDK Corp,0
139,supplier of,Apple Inc,LG Electronics Inc,0
142,supplier of,Apple Inc,Cowell e Holdings Inc,0
234,supplier of,Apple Inc,Coxon Precise Industrial Co Ltd,0
...,...,...,...,...
14669,supplier of,Apple Inc,Samsung Group,0
14670,supplier of,Apple Inc,Global Unichip Corp,0
14671,supplier of,Apple Inc,Electronic Arts Inc,0
14672,supplier of,Apple Inc,Pelephone Communications Ltd,0


In [19]:
corpus_split[corpus_split.entity1.astype(str).str.contains('Apple')]

,entity1,entity2,left,middle,right,article,label,sentences,split
0,Apple Inc,Samsung Electronics Co Ltd,These reviews have been especially popular wit...,(AAPL.O) and,(005930.KS) frequently hit with patent infrin...,50b8f1a67bb7c2cab2fe56b55a24f59ad1703c07b7053e...,0,These reviews have been especially popular wit...,0
21,Apple Inc,Amazon.com Inc,,shares lost {AAPL.O;-PCTCHNG:2} percent as wo...,"and Netflix, also weighed on the Nasdaq.",90ce5e433c2f36e596ecc077cc8d101da1e52fbbfe8f60...,0,Apple Inc shares lost {AAPL.O;-PCTCHNG:2} pe...,0
25,Apple Inc,Corning Inc,,shares lost {AAPL.O;-PCTCHNG:2} percent as wo...,reported a drop in screen glass sales for the...,90ce5e433c2f36e596ecc077cc8d101da1e52fbbfe8f60...,1,Apple Inc shares lost {AAPL.O;-PCTCHNG:2} pe...,0
26,Apple Inc,Netflix Inc,,shares lost {AAPL.O;-PCTCHNG:2} percent as wo...,", also weighed on the Nasdaq.",90ce5e433c2f36e596ecc077cc8d101da1e52fbbfe8f60...,0,Apple Inc shares lost {AAPL.O;-PCTCHNG:2} pe...,0
30,Apple Inc,Corning Inc,At least two other,suppliers - contract chipmaker Taiwan (2330.T...,said net sales in its specialty materials bus...,d70b8e93655d2bcfe4d17991e8d6a3a6ef5b8bd6b86d55...,1,At least two other Apple Inc suppliers - con...,0
...,...,...,...,...,...,...,...,...,...
86275,Apple Inc,Samsung Electronics Co Ltd,"""Xiaomi has a different business model than",or,","" said Ben Thompson, founder of Stratechery.c...",314f65ec6b5b415077196cf630f4610077090b206a215c...,0,"""Xiaomi has a different business model than A...",0
86282,Apple Inc,Huawei Technologies Co Ltd,But small players compete for just one-fifth o...,", Samsung, Lenovo Group,",[HWT.UL] and Xiaomi - according to Gartner.,f1adc11d5bb922a82100ef54cf12f8e155920e66870d1a...,1,But small players compete for just one-fifth o...,0
86283,Apple Inc,Samsung Electronics Co Ltd,But small players compete for just one-fifth o...,",",", Lenovo Group, Huawei [HWT.UL] and Xiaomi - a...",f1adc11d5bb922a82100ef54cf12f8e155920e66870d1a...,0,But small players compete for just one-fifth o...,0
86289,Apple Inc,Amazon.com Inc,The showing is a chance for Google and Microso...,",","and cable and satellite operators, to raise t...",c47e4f5ae80aca73dc67ea032a9ada5c96c1f1e1db103e...,0,The showing is a chance for Google and Microso...,0
